# LangChain 0.3 中的 LCEL 详细介绍


LCEL（LangChain Expression Language）是 LangChain 0.3 中的声明式编程框架，用于构建复杂的 AI 应用链。它基于 `Runnable` 接口，提供了强大的组合能力和优化的执行性能。

LCEL 核心概念

LCEL 的核心是 `Runnable` 接口，所有组件都实现了这个接口，支持：
- **同步/异步执行**：`invoke()` 和 `ainvoke()`
- **批量处理**：`batch()` 和 `abatch()`
- **流式处理**：`stream()` 和 `astream()`
- **并行执行**：`RunnableParallel`
- **条件分支**：`RunnableBranch`

In [1]:

## 完整代码示例

"""
LangChain 0.3 LCEL 完整示例集合
基于 LangChain 0.3.26 版本
"""

import asyncio
import json
from typing import Dict, List, Any, Optional
from datetime import datetime

# LangChain 核心组件
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableLambda,
    RunnableParallel,
    RunnableBranch,
    RunnableMap,
    RunnableSequence
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_ollama import OllamaLLM, ChatOllama

# 配置
OLLAMA_BASE_URL = "http://localhost:11434"
OLLAMA_MODEL = "qwen2.5:3b"

def create_llm():
    """创建LLM实例"""
    return OllamaLLM(
        base_url=OLLAMA_BASE_URL,
        model=OLLAMA_MODEL,
        temperature=0.7
    )

def create_chat_llm():
    """创建Chat LLM实例"""
    return ChatOllama(
        base_url=OLLAMA_BASE_URL,
        model=OLLAMA_MODEL,
        temperature=0.7
    )

## 1. 基础 LCEL 链组合

In [2]:

# 1. 基础 LCEL 链组合
def basic_chain_example():
    """基础链组合示例"""
    print("=" * 60)
    print("1. 基础 LCEL 链组合")
    print("=" * 60)

    llm = create_llm()

    # 创建提示模板
    prompt = PromptTemplate.from_template("请用中文回答：{question}")

    # 创建输出解析器
    output_parser = StrOutputParser()

    # 使用 pipe 方法创建链
    chain = prompt | llm | output_parser

    # 调用链
    result = chain.invoke({"question": "什么是人工智能？"})
    print(f"问题：什么是人工智能？")
    print(f"回答：{result}")

    return chain
basic_chain_example()


1. 基础 LCEL 链组合
问题：什么是人工智能？
回答：人工智能（Artificial Intelligence，简称AI）是指由人设计出的一套系统或算法，这套系统或算法能够模拟、扩展和增强人的智能。它让机器能够在没有人类明确编程的情况下学习、推理、解决问题以及完成各种任务。

简单来说，人工智能是一种技术，通过计算机程序和算法来使机器具备理解环境并执行特定任务的能力。这些任务可以包括语音识别、图像处理、语言翻译、玩游戏、自动驾驶汽车等，甚至还能进行创造性活动如艺术创作或音乐生成。

人工智能的核心在于让机器能够模仿人类的智能行为，包括学习能力、推理能力、问题解决能力和创造性的表达方式。它的发展目标是让计算机系统具备类人的认知和决策能力，以达到甚至超越人类的能力。


PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='请用中文回答：{question}')
| OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')
| StrOutputParser()

## 2. RunnablePassthrough 使用

In [3]:

def passthrough_example():
    """RunnablePassthrough 示例"""
    print("\n" + "=" * 60)
    print("2. RunnablePassthrough 使用")
    print("=" * 60)

    llm = create_llm()

    # 基础透传
    def format_docs(docs):
        return "\n\n".join([f"文档{i+1}: {doc}" for i, doc in enumerate(docs)])

    # 创建链，保留原始输入并添加格式化文档
    chain = (
        RunnablePassthrough.assign(
            formatted_docs=lambda x: format_docs(x["documents"])
        )
        | RunnablePassthrough.assign(
            prompt=lambda x: f"基于以下文档回答问题：\n{x['formatted_docs']}\n\n问题：{x['question']}"
        )
        | (lambda x: x["prompt"])
        | llm
        | StrOutputParser()
    )

    # 测试数据
    input_data = {
        "question": "什么是机器学习？",
        "documents": [
            "机器学习是人工智能的一个分支",
            "它通过算法让计算机从数据中学习",
            "常见的机器学习方法包括监督学习和无监督学习"
        ]
    }

    result = chain.invoke(input_data)
    print(f"问题：{input_data['question']}")
    print(f"回答：{result}")

    return chain
passthrough_example()


2. RunnablePassthrough 使用
问题：什么是机器学习？
回答：机器学习是人工智能的一个分支，它通过算法让计算机从数据中学习。常见的机器学习方法包括监督学习和无监督学习。


RunnableAssign(mapper={
  formatted_docs: RunnableLambda(lambda x: format_docs(x['documents']))
})
| RunnableAssign(mapper={
    prompt: RunnableLambda(lambda x: f"基于以下文档回答问题：\n{x['formatted_docs']}\n\n问题：{x['question']}")
  })
| RunnableLambda(...)
| OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')
| StrOutputParser()

## 3. RunnableParallel 并行处理

In [4]:
def parallel_example():
    """RunnableParallel 并行处理示例"""
    print("\n" + "=" * 60)
    print("3. RunnableParallel 并行处理")
    print("=" * 60)

    llm = create_llm()

    # 创建不同的分析提示
    sentiment_prompt = PromptTemplate.from_template("分析以下文本的情感（积极/消极/中性）：{text}")
    topic_prompt = PromptTemplate.from_template("提取以下文本的主要话题：{text}")
    summary_prompt = PromptTemplate.from_template("用一句话总结以下文本：{text}")

    # 创建并行分析链
    parallel_chain = RunnableParallel({
        "sentiment": sentiment_prompt | llm | StrOutputParser(),
        "topic": topic_prompt | llm | StrOutputParser(),
        "summary": summary_prompt | llm | StrOutputParser(),
        "original": RunnablePassthrough()
    })

    # 测试文本
    text = "今天天气真好，我和朋友们去公园玩了一整天，感觉非常开心和放松。"

    result = parallel_chain.invoke({"text": text})

    print(f"原文：{text}")
    print(f"情感分析：{result['sentiment']}")
    print(f"主题提取：{result['topic']}")
    print(f"文本摘要：{result['summary']}")

    return parallel_chain
parallel_example()


3. RunnableParallel 并行处理
原文：今天天气真好，我和朋友们去公园玩了一整天，感觉非常开心和放松。
情感分析：这个文本表达的是积极的情感。作者描述了美好的天气以及与朋友一起度过的一整天愉快时光，并且提到了自己感到很开心和放松的状态，这些都表明了积极的情绪体验。
主题提取：这个文本的主要话题是关于作者和他的朋友们在公园度过的一天的愉快经历，重点在于天气良好以及他们感到高兴和放松的状态。主要话题可以总结为“户外休闲与愉悦”。
文本摘要：今天在公园度过的一天因为好天气而格外愉快和放松。


{
  sentiment: PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='分析以下文本的情感（积极/消极/中性）：{text}')
             | OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')
             | StrOutputParser(),
  topic: PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='提取以下文本的主要话题：{text}')
         | OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')
         | StrOutputParser(),
  summary: PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='用一句话总结以下文本：{text}')
           | OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')
           | StrOutputParser(),
  original: RunnablePassthrough()
}

## 4. RunnableBranch 条件分支

In [5]:
def branch_example():
    """RunnableBranch 条件分支示例"""
    print("\n" + "=" * 60)
    print("4. RunnableBranch 条件分支")
    print("=" * 60)

    llm = create_llm()

    # 定义条件函数
    def is_question(x):
        return x["text"].strip().endswith("？") or x["text"].strip().endswith("?")

    def is_greeting(x):
        greetings = ["你好", "hello", "hi", "早上好", "晚上好"]
        return any(greeting in x["text"].lower() for greeting in greetings)

    # 创建不同的处理链
    question_chain = PromptTemplate.from_template("请详细回答这个问题：{text}") | llm
    greeting_chain = PromptTemplate.from_template("友好地回应这个问候：{text}") | llm
    default_chain = PromptTemplate.from_template("请对以下内容进行评论：{text}") | llm

    # 创建分支链
    branch_chain = RunnableBranch(
        (is_question, question_chain),
        (is_greeting, greeting_chain),
        default_chain
    ) | StrOutputParser()

    # 测试不同类型的输入
    test_inputs = [
        {"text": "你好！"},
        {"text": "什么是深度学习？"},
        {"text": "今天天气不错"}
    ]

    for i, input_data in enumerate(test_inputs, 1):
        result = branch_chain.invoke(input_data)
        print(f"输入{i}：{input_data['text']}")
        print(f"输出{i}：{result}\n")

    return branch_chain
branch_example()



4. RunnableBranch 条件分支
输入1：你好！
输出1：你好！很高兴能帮助你。有什么我可以为你效劳的吗？

输入2：什么是深度学习？
输出2：深度学习是一种机器学习的子集，它允许计算机通过构建和训练多层神经网络来自动从大量数据中提取抽象特征。这些层次化的结构模仿了人脑处理信息的方式，即每个“层”可以识别越来越高级别的模式或特征。

在深度学习模型中，输入的数据首先被传递到最底层的神经元，这些神经元会对数据进行初步的非线性变换，从而发现更复杂的、与目标相关的信息。然后，这个结果被传递给下一个层次的神经元，直到达到顶部，即输出层。这种结构被称为多层感知器（Multilayer Perceptron, MLP）。

深度学习在训练过程中使用大量的标注数据来优化其模型参数，以提高预测准确率和泛化能力。常用的深度学习模型包括卷积神经网络（Convolutional Neural Networks, CNNs）、循环神经网络（Recurrent Neural Networks, RNNs）和生成对抗网络（Generative Adversarial Networks, GANs），其中最出名的是前两者。

深度学习已经成为许多领域的关键技术，如自然语言处理、图像识别、语音识别以及预测分析等。通过使用大量的计算资源和高效的数据预处理技术来加速模型训练过程，深度学习已经在这些领域取得了显著的成果，并且正在持续发展之中。

输入3：今天天气不错
输出3：今天的天气确实不错，这通常意味着是一个适宜外出、享受户外活动的好日子。不过具体的感受还会受到其他因素的影响，比如温度、湿度和个人喜好等。总的来说，这是一个令人愉悦的天气条件。如果您有特定地点或具体的要求，请告诉我！



RunnableBranch(branches=[(RunnableLambda(is_question), PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='请详细回答这个问题：{text}')
| OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434')), (RunnableLambda(is_greeting), PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='友好地回应这个问候：{text}')
| OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434'))], default=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='请对以下内容进行评论：{text}')
| OllamaLLM(model='qwen2.5:3b', temperature=0.7, base_url='http://localhost:11434'))
| StrOutputParser()

## 5. RunnableLambda 自定义函数

In [ ]:
def lambda_example():
    """RunnableLambda 自定义函数示例"""
    print("\n" + "=" * 60)
    print("5. RunnableLambda 自定义函数")
    print("=" * 60)

    llm = create_llm()

    # 自定义处理函数
    def preprocess_text(x):
        """文本预处理"""
        text = x["text"]
        # 清理文本
        text = text.strip()
        # 添加元数据
        return {
            "processed_text": text,
            "word_count": len(text.split()),
            "char_count": len(text),
            "timestamp": datetime.now().isoformat()
        }

    def postprocess_result(x):
        """结果后处理"""
        return {
            "response": x,
            "processed_at": datetime.now().isoformat(),
            "response_length": len(x)
        }

    # 创建包含自定义函数的链
    chain = (
        RunnableLambda(preprocess_text)
        | RunnablePassthrough.assign(
            ai_response=lambda x: (
                PromptTemplate.from_template("请分析以下文本（{word_count}词，{char_count}字符）：{processed_text}")
                | llm
                | StrOutputParser()
            ).invoke(x)
        )
        | RunnableLambda(lambda x: postprocess_result(x["ai_response"]))
    )

    # 测试
    input_text = {"text": "人工智能正在改变我们的生活方式，从智能手机到自动驾驶汽车。"}
    result = chain.invoke(input_text)

    print(f"输入：{input_text['text']}")
    print(f"AI回应：{result['response']}")
    print(f"处理时间：{result['processed_at']}")
    print(f"回应长度：{result['response_length']}字符")

    return chain
lambda_example()

## 6. 带记忆的对话链

In [ ]:
def memory_chat_example():
    """带记忆的对话链示例"""
    print("\n" + "=" * 60)
    print("6. 带记忆的对话链")
    print("=" * 60)

    chat_llm = create_chat_llm()

    # 创建聊天提示模板
    prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一个友好的AI助手，能够记住之前的对话内容。"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])

    # 创建基础链
    chain = prompt | chat_llm | StrOutputParser()

    # 创建记忆存储
    memory = InMemoryChatMessageHistory()

    # 创建带记忆的链
    chain_with_memory = RunnableWithMessageHistory(
        chain,
        lambda session_id: memory,
        input_messages_key="input",
        history_messages_key="history"
    )

    # 模拟对话
    conversations = [
        "我叫张三，今年25岁",
        "我的爱好是什么？",
        "我多大了？",
        "能总结一下我们的对话吗？"
    ]

    config = {"configurable": {"session_id": "user_123"}}

    for i, message in enumerate(conversations, 1):
        response = chain_with_memory.invoke(
            {"input": message},
            config=config
        )
        print(f"用户{i}：{message}")
        print(f"AI{i}：{response}\n")

    return chain_with_memory

## 7. 复杂数据处理链

In [ ]:
def complex_data_processing():
    """复杂数据处理链示例"""
    print("\n" + "=" * 60)
    print("7. 复杂数据处理链")
    print("=" * 60)

    llm = create_llm()

    # 模拟数据
    data = {
        "users": [
            {"name": "张三", "age": 25, "city": "北京", "interests": ["编程", "阅读"]},
            {"name": "李四", "age": 30, "city": "上海", "interests": ["音乐", "旅行"]},
            {"name": "王五", "age": 28, "city": "深圳", "interests": ["运动", "摄影"]}
        ],
        "query": "分析用户群体特征"
    }

    # 数据处理函数
    def analyze_users(x):
        users = x["users"]
        total_users = len(users)
        avg_age = sum(user["age"] for user in users) / total_users
        cities = list(set(user["city"] for user in users))
        all_interests = []
        for user in users:
            all_interests.extend(user["interests"])

        return {
            "total_users": total_users,
            "average_age": round(avg_age, 1),
            "cities": cities,
            "common_interests": list(set(all_interests)),
            "original_query": x["query"]
        }

    def format_analysis(x):
        return f"""
用户群体分析报告：
- 总用户数：{x['total_users']}人
- 平均年龄：{x['average_age']}岁
- 分布城市：{', '.join(x['cities'])}
- 兴趣爱好：{', '.join(x['common_interests'])}

请基于以上数据回答：{x['original_query']}
"""

    # 创建处理链
    processing_chain = (
        RunnableLambda(analyze_users)
        | RunnableLambda(format_analysis)
        | llm
        | StrOutputParser()
    )

    result = processing_chain.invoke(data)
    print("数据分析结果：")
    print(result)

    return processing_chain

## 8. 异步处理示例

In [ ]:
async def async_example():
    """异步处理示例"""
    print("\n" + "=" * 60)
    print("8. 异步处理示例")
    print("=" * 60)

    llm = create_llm()

    # 创建异步处理链
    prompt = PromptTemplate.from_template("请用中文简要回答：{question}")
    chain = prompt | llm | StrOutputParser()

    # 准备多个问题
    questions = [
        {"question": "什么是机器学习？"},
        {"question": "什么是深度学习？"},
        {"question": "什么是自然语言处理？"},
        {"question": "什么是计算机视觉？"}
    ]

    print("开始异步处理多个问题...")
    start_time = datetime.now()

    # 异步批量处理
    results = await chain.abatch(questions)

    end_time = datetime.now()
    processing_time = (end_time - start_time).total_seconds()

    print(f"处理完成，耗时：{processing_time:.2f}秒\n")

    for i, (question, result) in enumerate(zip(questions, results), 1):
        print(f"问题{i}：{question['question']}")
        print(f"回答{i}：{result}\n")

    return chain

## 9. 流式处理示例

In [ ]:
def streaming_example():
    """流式处理示例"""
    print("\n" + "=" * 60)
    print("9. 流式处理示例")
    print("=" * 60)

    llm = create_llm()

    prompt = PromptTemplate.from_template("请详细解释：{topic}")
    chain = prompt | llm | StrOutputParser()

    print("开始流式生成回答...")
    print("问题：什么是人工智能？")
    print("回答：", end="", flush=True)

    # 流式处理
    for chunk in chain.stream({"topic": "什么是人工智能"}):
        print(chunk, end="", flush=True)

    print("\n\n流式处理完成！")

    return chain

## 10. JSON 输出解析示例

In [ ]:
def json_output_example():
    """JSON输出解析示例"""
    print("\n" + "=" * 60)
    print("10. JSON输出解析示例")
    print("=" * 60)

    llm = create_llm()

    # 创建JSON输出解析器
    json_parser = JsonOutputParser()

    # 创建提示模板，要求JSON格式输出
    prompt = PromptTemplate.from_template("""
请分析以下文本并以JSON格式返回结果：
文本：{text}

请返回包含以下字段的JSON：
- sentiment: 情感（positive/negative/neutral）
- topics: 主要话题列表
- summary: 一句话总结
- word_count: 词数

JSON格式：
""")

    # 创建链
    chain = prompt | llm | json_parser

    # 测试文本
    test_text = "今天的会议非常成功，我们讨论了新产品的开发计划和市场策略，团队成员都很积极参与。"

    try:
        result = chain.invoke({"text": test_text})
        print(f"输入文本：{test_text}")
        print("JSON分析结果：")
        print(json.dumps(result, ensure_ascii=False, indent=2))
    except Exception as e:
        print(f"JSON解析失败：{e}")
        # 降级处理
        simple_chain = prompt | llm | StrOutputParser()
        result = simple_chain.invoke({"text": test_text})
        print(f"文本结果：{result}")

    return chain

In [ ]:

# ============================================================================
# 主函数
# ============================================================================

def main():
    """运行所有示例"""
    print("LangChain 0.3 LCEL 完整示例集合")
    print("基于 LangChain 0.3.26 版本")
    print("确保 Ollama 服务正在运行：http://localhost:11434")

    try:
        # 运行所有同步示例
        basic_chain_example()
        passthrough_example()
        parallel_example()
        branch_example()
        lambda_example()
        memory_chat_example()
        complex_data_processing()
        streaming_example()
        json_output_example()

        # 运行异步示例
        print("\n开始运行异步示例...")
        asyncio.run(async_example())

        print("\n" + "=" * 60)
        print("所有示例运行完成！")
        print("=" * 60)

    except Exception as e:
        print(f"运行出错：{e}")
        print("请确保：")
        print("1. Ollama 服务正在运行")
        print("2. qwen2.5:3b 模型已下载")
        print("3. 网络连接正常")

if __name__ == "__main__":
    main()


# 总结

LCEL 是 LangChain 0.3 中的核心特性，提供了：

1. **声明式编程**：简洁的链式语法
2. **强大组合能力**：灵活的组件组合
3. **性能优化**：自动并行化和批处理
4. **流式支持**：实时响应能力
5. **调试友好**：清晰的执行流程

**选择建议**：
- 简单应用：使用基础链组合
- 复杂逻辑：使用 `RunnableBranch` 和 `RunnableParallel`
- 高性能需求：利用批处理和并行特性
- 复杂状态管理：考虑升级到 LangGraph

所有示例代码都基于 LangChain 0.3.26 版本，确保与您的环境兼容。


---

In [1]:
"""
LangChain 0.3 LCEL 高级示例集合
基于 LangChain 0.3.26 版本
"""

import asyncio
import json
import time
from typing import Dict, List, Any, Optional, Union
from datetime import datetime
from collections import deque

# LangChain 核心组件
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableLambda,
    RunnableParallel,
    RunnableBranch,
    RunnableMap,
    RunnableSequence,
    RunnableConfig,
    Runnable
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_ollama import OllamaLLM, ChatOllama
from pydantic import BaseModel, Field

In [ ]:

# 配置
OLLAMA_BASE_URL = "http://localhost:11434"
OLLAMA_MODEL = "qwen2.5:3b"

def create_llm():
    """创建LLM实例"""
    return OllamaLLM(
        base_url=OLLAMA_BASE_URL,
        model=OLLAMA_MODEL,
        temperature=0.7
    )

def create_chat_llm():
    """创建Chat LLM实例"""
    return ChatOllama(
        base_url=OLLAMA_BASE_URL,
        model=OLLAMA_MODEL,
        temperature=0.7
    )

# ============================================================================
# 1. 基础 LCEL 操作符示例
# ============================================================================

def basic_operators_example():
    """基础 LCEL 操作符示例"""
    print("=" * 60)
    print("1. 基础 LCEL 操作符")
    print("=" * 60)

    llm = create_llm()

    # 管道操作符 |
    prompt = PromptTemplate.from_template("翻译成英文：{text}")
    chain1 = prompt | llm | StrOutputParser()

    # 等价于 RunnableSequence
    chain2 = RunnableSequence(first=prompt, middle=[llm], last=StrOutputParser())

    result1 = chain1.invoke({"text": "你好世界"})
    result2 = chain2.invoke({"text": "你好世界"})

    print(f"管道操作符结果：{result1}")
    print(f"RunnableSequence结果：{result2}")

# ============================================================================
# 2. RunnablePassthrough 高级用法
# ============================================================================

def advanced_passthrough_example():
    """RunnablePassthrough 高级用法"""
    print("\n" + "=" * 60)
    print("2. RunnablePassthrough 高级用法")
    print("=" * 60)

    llm = create_llm()

    # 使用 assign 添加新字段
    def calculate_stats(x):
        text = x["text"]
        return {
            "word_count": len(text.split()),
            "char_count": len(text),
            "has_question": "?" in text or "？" in text
        }

    # 复杂的数据流处理
    chain = (
        RunnablePassthrough.assign(stats=RunnableLambda(calculate_stats))
        | RunnablePassthrough.assign(
            analysis_prompt=lambda x: f"""
分析以下文本（{x['stats']['word_count']}词，{x['stats']['char_count']}字符）：
文本：{x['text']}
是否包含问题：{x['stats']['has_question']}

请提供详细分析：
"""
        )
        | RunnablePassthrough.assign(
            analysis=lambda x: (PromptTemplate.from_template("{analysis_prompt}") | llm | StrOutputParser()).invoke(x)
        )
        | RunnableLambda(lambda x: {
            "original": x["text"],
            "stats": x["stats"],
            "analysis": x["analysis"]
        })
    )

    result = chain.invoke({"text": "人工智能的发展前景如何？它会改变我们的生活吗？"})

    print(f"原文：{result['original']}")
    print(f"统计：{result['stats']}")
    print(f"分析：{result['analysis']}")

# ============================================================================
# 3. RunnableParallel 复杂并行处理
# ============================================================================

def complex_parallel_example():
    """复杂并行处理示例"""
    print("\n" + "=" * 60)
    print("3. 复杂并行处理")
    print("=" * 60)

    llm = create_llm()

    # 定义多个分析任务
    sentiment_chain = (
        PromptTemplate.from_template("分析情感（积极/消极/中性）：{text}")
        | llm | StrOutputParser()
    )

    topic_chain = (
        PromptTemplate.from_template("提取3个主要关键词：{text}")
        | llm | StrOutputParser()
    )

    summary_chain = (
        PromptTemplate.from_template("一句话总结：{text}")
        | llm | StrOutputParser()
    )

    length_analysis = RunnableLambda(lambda x: {
        "word_count": len(x["text"].split()),
        "char_count": len(x["text"]),
        "sentence_count": len([s for s in x["text"].split("。") if s.strip()])
    })

    # 创建复杂并行链
    parallel_chain = RunnableParallel({
        "sentiment": sentiment_chain,
        "topics": topic_chain,
        "summary": summary_chain,
        "length_stats": length_analysis,
        "original": RunnablePassthrough(),
        "timestamp": RunnableLambda(lambda x: datetime.now().isoformat())
    })

    # 后处理：合并结果
    def format_results(results):
        return f"""
文本分析报告
================
原文：{results['original']['text']}
时间：{results['timestamp']}

情感分析：{results['sentiment']}
关键词：{results['topics']}
摘要：{results['summary']}

统计信息：
- 字数：{results['length_stats']['word_count']}
- 字符数：{results['length_stats']['char_count']}
- 句子数：{results['length_stats']['sentence_count']}
"""

    final_chain = parallel_chain | RunnableLambda(format_results)

    text = "今天参加了一个关于人工智能的会议，讨论了机器学习、深度学习和自然语言处理的最新进展。专家们分享了很多有趣的观点，让我对AI的未来发展有了更深的理解。"

    result = final_chain.invoke({"text": text})
    print(result)

# ============================================================================
# 4. RunnableBranch 复杂条件分支
# ============================================================================

def complex_branch_example():
    """复杂条件分支示例"""
    print("\n" + "=" * 60)
    print("4. 复杂条件分支")
    print("=" * 60)

    llm = create_llm()

    # 复杂条件判断函数
    def is_technical_question(x):
        technical_keywords = ["算法", "编程", "代码", "技术", "开发", "API", "数据库"]
        return any(keyword in x["text"] for keyword in technical_keywords)

    def is_business_question(x):
        business_keywords = ["市场", "销售", "客户", "收入", "成本", "利润", "商业"]
        return any(keyword in x["text"] for keyword in business_keywords)

    def is_personal_question(x):
        personal_keywords = ["我", "个人", "建议", "帮助", "怎么办"]
        return any(keyword in x["text"] for keyword in personal_keywords)

    def get_text_complexity(x):
        text = x["text"]
        return len(text.split()) > 20  # 超过20词认为是复杂问题

    # 创建不同类型的处理链
    technical_chain = (
        PromptTemplate.from_template("""
作为技术专家，请详细回答这个技术问题：{text}

请包括：
1. 技术原理
2. 实现方法
3. 最佳实践
""")
        | llm | StrOutputParser()
    )

    business_chain = (
        PromptTemplate.from_template("""
作为商业顾问，请分析这个商业问题：{text}

请包括：
1. 市场分析
2. 风险评估
3. 建议方案
""")
        | llm | StrOutputParser()
    )

    personal_chain = (
        PromptTemplate.from_template("""
作为生活顾问，请给出贴心的建议：{text}

请提供：
1. 理解和共情
2. 具体建议
3. 鼓励话语
""")
        | llm | StrOutputParser()
    )

    # 复杂问题需要更详细的分析
    complex_analysis_chain = (
        RunnablePassthrough.assign(
            analysis=lambda x: (
                PromptTemplate.from_template("首先分析问题的核心要点：{text}")
                | llm | StrOutputParser()
            ).invoke(x)
        )
        | RunnablePassthrough.assign(
            detailed_response=lambda x: (
                PromptTemplate.from_template("""
基于分析：{analysis}

请详细回答原问题：{text}
""")
                | llm | StrOutputParser()
            ).invoke(x)
        )
        | RunnableLambda(lambda x: f"分析：{x['analysis']}\n\n详细回答：{x['detailed_response']}")
    )

    simple_chain = (
        PromptTemplate.from_template("简洁回答：{text}")
        | llm | StrOutputParser()
    )

    # 创建嵌套分支
    complexity_branch = RunnableBranch(
        (get_text_complexity, complex_analysis_chain),
        simple_chain
    )

    main_branch = RunnableBranch(
        (is_technical_question, technical_chain),
        (is_business_question, business_chain),
        (is_personal_question, personal_chain),
        complexity_branch  # 默认根据复杂度处理
    )

    # 测试不同类型的问题
    test_questions = [
        {"text": "如何实现一个高效的排序算法？"},
        {"text": "我们公司的市场策略应该如何调整？"},
        {"text": "我最近工作压力很大，该怎么办？"},
        {"text": "什么是人工智能？"},
        {"text": "请详细解释深度学习的工作原理，包括反向传播算法、梯度下降优化、正则化技术，以及在计算机视觉和自然语言处理中的具体应用案例。"}
    ]

    for i, question in enumerate(test_questions, 1):
        print(f"\n问题{i}：{question['text']}")
        result = main_branch.invoke(question)
        print(f"回答{i}：{result}")
        print("-" * 40)

# ============================================================================
# 5. 自定义 Runnable 类
# ============================================================================

class CustomTextProcessor(Runnable):
    """自定义文本处理器"""

    def __init__(self, processing_type: str = "default"):
        self.processing_type = processing_type

    def invoke(self, input: Dict[str, Any], config: Optional[RunnableConfig] = None) -> Dict[str, Any]:
        text = input.get("text", "")

        if self.processing_type == "uppercase":
            processed = text.upper()
        elif self.processing_type == "reverse":
            processed = text[::-1]
        elif self.processing_type == "word_count":
            processed = f"字数统计：{len(text.split())}词"
        else:
            processed = text.strip()

        return {
            "original": text,
            "processed": processed,
            "type": self.processing_type,
            "timestamp": datetime.now().isoformat()
        }

def custom_runnable_example():
    """自定义 Runnable 示例"""
    print("\n" + "=" * 60)
    print("5. 自定义 Runnable 类")
    print("=" * 60)

    # 创建不同类型的处理器
    processors = {
        "uppercase": CustomTextProcessor("uppercase"),
        "reverse": CustomTextProcessor("reverse"),
        "word_count": CustomTextProcessor("word_count")
    }

    # 创建并行处理链
    parallel_processing = RunnableParallel({
        "uppercase_result": processors["uppercase"],
        "reverse_result": processors["reverse"],
        "count_result": processors["word_count"],
        "original": RunnablePassthrough()
    })

    test_text = {"text": "Hello LangChain LCEL"}
    result = parallel_processing.invoke(test_text)

    print(f"原文：{result['original']['text']}")
    print(f"大写处理：{result['uppercase_result']['processed']}")
    print(f"反转处理：{result['reverse_result']['processed']}")
    print(f"计数处理：{result['count_result']['processed']}")

# ============================================================================
# 6. 流式处理高级示例
# ============================================================================

def advanced_streaming_example():
    """高级流式处理示例"""
    print("\n" + "=" * 60)
    print("6. 高级流式处理")
    print("=" * 60)

    llm = create_llm()

    # 创建流式处理链
    prompt = PromptTemplate.from_template("""
请详细解释以下概念，并给出实际应用例子：{topic}

请按以下结构回答：
1. 定义和基本概念
2. 核心原理
3. 实际应用
4. 发展趋势
""")

    # 添加流式处理的元数据
    def add_streaming_metadata(chunk):
        return {
            "content": chunk,
            "timestamp": datetime.now().isoformat(),
            "chunk_length": len(chunk)
        }

    streaming_chain = (
        prompt
        | llm
        | StrOutputParser()
        | RunnableLambda(add_streaming_metadata)
    )

    print("开始流式生成（带元数据）...")
    print("主题：机器学习")
    print("\n回答：")

    total_chunks = 0
    total_length = 0

    for chunk_data in streaming_chain.stream({"topic": "机器学习"}):
        content = chunk_data["content"]
        print(content, end="", flush=True)
        total_chunks += 1
        total_length += chunk_data["chunk_length"]

    print(f"\n\n流式处理完成！")
    print(f"总块数：{total_chunks}")
    print(f"总长度：{total_length}字符")

# ============================================================================
# 7. 批量处理示例
# ============================================================================

async def batch_processing_example():
    """批量处理示例"""
    print("\n" + "=" * 60)
    print("7. 批量处理示例")
    print("=" * 60)

    llm = create_llm()

    # 创建批量处理链
    prompt = PromptTemplate.from_template("用一句话解释：{concept}")
    chain = prompt | llm | StrOutputParser()

    # 准备批量数据
    concepts = [
        {"concept": "人工智能"},
        {"concept": "机器学习"},
        {"concept": "深度学习"},
        {"concept": "自然语言处理"},
        {"concept": "计算机视觉"},
        {"concept": "强化学习"}
    ]

    print("开始批量处理...")
    start_time = time.time()

    # 同步批量处理
    sync_results = chain.batch(concepts)
    sync_time = time.time() - start_time

    print(f"同步批量处理完成，耗时：{sync_time:.2f}秒")

    # 异步批量处理
    start_time = time.time()
    async_results = await chain.abatch(concepts)
    async_time = time.time() - start_time

    print(f"异步批量处理完成，耗时：{async_time:.2f}秒")

    # 显示结果
    for i, (concept, sync_result, async_result) in enumerate(zip(concepts, sync_results, async_results), 1):
        print(f"\n概念{i}：{concept['concept']}")
        print(f"同步结果：{sync_result}")
        print(f"异步结果：{async_result}")

# ============================================================================
# 8. 错误处理和重试机制
# ============================================================================

def error_handling_example():
    """错误处理和重试机制示例"""
    print("\n" + "=" * 60)
    print("8. 错误处理和重试机制")
    print("=" * 60)

    llm = create_llm()

    # 模拟可能失败的处理函数
    def risky_processing(x):
        import random
        if random.random() < 0.3:  # 30% 失败率
            raise Exception("模拟处理失败")
        return {"processed": f"成功处理：{x['text']}", "status": "success"}

    # 重试装饰器
    def with_retry(func, max_retries=3):
        def wrapper(x):
            for attempt in range(max_retries):
                try:
                    return func(x)
                except Exception as e:
                    if attempt == max_retries - 1:
                        return {"error": str(e), "status": "failed", "attempts": attempt + 1}
                    print(f"尝试 {attempt + 1} 失败：{e}")
            return {"error": "最大重试次数已达到", "status": "failed"}
        return wrapper

    # 创建带错误处理的链
    safe_processing = RunnableLambda(with_retry(risky_processing))

    # 备用处理链
    fallback_chain = RunnableLambda(lambda x: {
        "processed": f"备用处理：{x['text']}",
        "status": "fallback"
    })

    # 主处理链
    main_chain = (
        RunnablePassthrough.assign(result=safe_processing)
        | RunnableLambda(lambda x:
            x["result"] if x["result"]["status"] != "failed"
            else {**x, "result": fallback_chain.invoke(x)}
        )
    )

    # 测试错误处理
    test_inputs = [
        {"text": "测试文本1"},
        {"text": "测试文本2"},
        {"text": "测试文本3"},
        {"text": "测试文本4"},
        {"text": "测试文本5"}
    ]

    for i, input_data in enumerate(test_inputs, 1):
        result = main_chain.invoke(input_data)
        print(f"输入{i}：{input_data['text']}")
        print(f"结果{i}：{result['result']}")
        print(f"状态{i}：{result['result']['status']}")
        print("-" * 30)

# ============================================================================
# 9. 动态链构建
# ============================================================================

def dynamic_chain_example():
    """动态链构建示例"""
    print("\n" + "=" * 60)
    print("9. 动态链构建")
    print("=" * 60)

    llm = create_llm()

    def build_dynamic_chain(processing_steps: List[str]):
        """根据配置动态构建处理链"""

        # 定义可用的处理步骤
        available_steps = {
            "sentiment": PromptTemplate.from_template("分析情感：{text}") | llm | StrOutputParser(),
            "summary": PromptTemplate.from_template("总结：{text}") | llm | StrOutputParser(),
            "translate": PromptTemplate.from_template("翻译成英文：{text}") | llm | StrOutputParser(),
            "keywords": PromptTemplate.from_template("提取关键词：{text}") | llm | StrOutputParser(),
            "length": RunnableLambda(lambda x: f"长度：{len(x['text'])}字符")
        }

        # 构建并行处理
        parallel_steps = {}
        for step in processing_steps:
            if step in available_steps:
                parallel_steps[step] = available_steps[step]

        if not parallel_steps:
            return RunnableLambda(lambda x: {"error": "没有有效的处理步骤"})

        # 添加原文
        parallel_steps["original"] = RunnablePassthrough()

        return RunnableParallel(parallel_steps)

    # 测试不同的配置
    configurations = [
        ["sentiment", "summary"],
        ["translate", "keywords", "length"],
        ["sentiment", "summary", "translate", "keywords"],
        ["invalid_step"]  # 测试错误情况
    ]

    test_text = {"text": "今天天气很好，我和朋友们去公园散步，感觉很放松。"}

    for i, config in enumerate(configurations, 1):
        print(f"\n配置{i}：{config}")
        chain = build_dynamic_chain(config)
        result = chain.invoke(test_text)

        for key, value in result.items():
            if key != "original":
                print(f"  {key}: {value}")

# ============================================================================
# 10. 复杂数据流处理
# ============================================================================

def complex_data_flow_example():
    """复杂数据流处理示例"""
    print("\n" + "=" * 60)
    print("10. 复杂数据流处理")
    print("=" * 60)

    llm = create_llm()

    # 模拟复杂的业务数据
    business_data = {
        "company": "TechCorp",
        "quarter": "Q3 2024",
        "revenue": 1500000,
        "expenses": 1200000,
        "employees": 150,
        "products": [
            {"name": "AI助手", "sales": 800000, "growth": 0.25},
            {"name": "数据分析平台", "sales": 500000, "growth": 0.15},
            {"name": "云服务", "sales": 200000, "growth": 0.35}
        ],
        "regions": {
            "北美": {"revenue": 600000, "growth": 0.20},
            "欧洲": {"revenue": 500000, "growth": 0.18},
            "亚洲": {"revenue": 400000, "growth": 0.30}
        }
    }

    # 数据预处理
    def calculate_metrics(data):
        profit = data["revenue"] - data["expenses"]
        profit_margin = profit / data["revenue"] * 100
        revenue_per_employee = data["revenue"] / data["employees"]

        # 产品分析
        best_product = max(data["products"], key=lambda x: x["sales"])
        fastest_growing = max(data["products"], key=lambda x: x["growth"])

        # 地区分析
        best_region = max(data["regions"].items(), key=lambda x: x[1]["revenue"])

        return {
            "financial_metrics": {
                "profit": profit,
                "profit_margin": round(profit_margin, 2),
                "revenue_per_employee": round(revenue_per_employee, 2)
            },
            "product_insights": {
                "best_selling": best_product["name"],
                "fastest_growing": fastest_growing["name"]
            },
            "regional_insights": {
                "top_region": best_region[0],
                "top_region_revenue": best_region[1]["revenue"]
            },
            "original_data": data
        }

    # 创建分析报告
    def generate_analysis_prompt(metrics):
        return f"""
请分析以下业务数据并生成专业报告：

公司：{metrics['original_data']['company']}
季度：{metrics['original_data']['quarter']}

财务指标：
- 收入：${metrics['original_data']['revenue']:,}
- 支出：${metrics['original_data']['expenses']:,}
- 利润：${metrics['financial_metrics']['profit']:,}
- 利润率：{metrics['financial_metrics']['profit_margin']}%
- 人均收入：${metrics['financial_metrics']['revenue_per_employee']:,}

产品表现：
- 最佳销售产品：{metrics['product_insights']['best_selling']}
- 增长最快产品：{metrics['product_insights']['fastest_growing']}

地区表现：
- 最佳地区：{metrics['regional_insights']['top_region']}
- 该地区收入：${metrics['regional_insights']['top_region_revenue']:,}

请提供：
1. 整体业务健康度评估
2. 关键优势和风险点
3. 改进建议
"""

    # 构建复杂处理链
    analysis_chain = (
        RunnableLambda(calculate_metrics)
        | RunnablePassthrough.assign(
            analysis_prompt=RunnableLambda(generate_analysis_prompt)
        )
        | RunnablePassthrough.assign(
            business_analysis=lambda x: (
                PromptTemplate.from_template("{analysis_prompt}")
                | llm | StrOutputParser()
            ).invoke(x)
        )
        | RunnablePassthrough.assign(
            executive_summary=lambda x: (
                PromptTemplate.from_template("""
基于以下分析，写一份执行摘要（不超过100字）：
{business_analysis}
""")
                | llm | StrOutputParser()
            ).invoke(x)
        )
        | RunnableLambda(lambda x: {
            "company": x["original_data"]["company"],
            "quarter": x["original_data"]["quarter"],
            "key_metrics": x["financial_metrics"],
            "insights": {
                "products": x["product_insights"],
                "regions": x["regional_insights"]
            },
            "detailed_analysis": x["business_analysis"],
            "executive_summary": x["executive_summary"],
            "generated_at": datetime.now().isoformat()
        })
    )

    # 执行分析
    result = analysis_chain.invoke(business_data)

    print(f"公司：{result['company']}")
    print(f"季度：{result['quarter']}")
    print(f"生成时间：{result['generated_at']}")
    print("\n关键指标：")
    for key, value in result['key_metrics'].items():
        print(f"  {key}: {value}")

    print("\n业务洞察：")
    print(f"  最佳产品：{result['insights']['products']['best_selling']}")
    print(f"  增长最快：{result['insights']['products']['fastest_growing']}")
    print(f"  最佳地区：{result['insights']['regions']['top_region']}")

    print(f"\n执行摘要：\n{result['executive_summary']}")
    print(f"\n详细分析：\n{result['detailed_analysis']}")

# ============================================================================
# 主函数
# ============================================================================

async def main():
    """运行所有示例"""
    print("LangChain 0.3 LCEL 高级示例集合")
    print("基于 LangChain 0.3.26 版本")
    print("确保 Ollama 服务正在运行：http://localhost:11434")

    try:
        # 运行所有同步示例
        basic_operators_example()
        advanced_passthrough_example()
        complex_parallel_example()
        complex_branch_example()
        custom_runnable_example()
        advanced_streaming_example()
        error_handling_example()
        dynamic_chain_example()
        complex_data_flow_example()

        # 运行异步示例
        print("\n开始运行异步示例...")
        await batch_processing_example()

        print("\n" + "=" * 60)
        print("所有高级示例运行完成！")
        print("=" * 60)

    except Exception as e:
        print(f"运行出错：{e}")
        print("请确保：")
        print("1. Ollama 服务正在运行")
        print("2. qwen2.5:3b 模型已下载")
        print("3. 网络连接正常")

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import uuid

"""
LCEL 专业应用场景示例
"""

# ============================================================================
# 11. RAG（检索增强生成）系统
# ============================================================================

def rag_system_example():
    """RAG 系统示例"""
    print("\n" + "=" * 60)
    print("11. RAG 检索增强生成系统")
    print("=" * 60)

    llm = create_llm()

    # 模拟文档数据库
    documents = {
        "doc1": "人工智能是计算机科学的一个分支，致力于创建能够执行通常需要人类智能的任务的系统。",
        "doc2": "机器学习是人工智能的一个子集，它使计算机能够在没有明确编程的情况下学习和改进。",
        "doc3": "深度学习是机器学习的一个分支，使用神经网络来模拟人脑的工作方式。",
        "doc4": "自然语言处理是人工智能的一个领域，专注于计算机与人类语言之间的交互。",
        "doc5": "计算机视觉是人工智能的一个分支，致力于让计算机能够理解和解释视觉信息。"
    }

    # 简单的检索函数
    def retrieve_documents(query: str, top_k: int = 3):
        """基于关键词匹配检索文档"""
        scores = {}
        query_words = query.lower().split()

        for doc_id, content in documents.items():
            score = sum(1 for word in query_words if word in content.lower())
            if score > 0:
                scores[doc_id] = score

        # 返回得分最高的文档
        sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        return [documents[doc_id] for doc_id, _ in sorted_docs[:top_k]]

    # 构建 RAG 链
    retrieval_chain = RunnableLambda(lambda x: {
        "query": x["query"],
        "retrieved_docs": retrieve_documents(x["query"])
    })

    context_chain = RunnableLambda(lambda x: {
        "query": x["query"],
        "context": "\n\n".join([f"文档{i+1}: {doc}" for i, doc in enumerate(x["retrieved_docs"])])
    })

    rag_prompt = PromptTemplate.from_template("""
基于以下文档回答问题：

{context}

问题：{query}

请基于提供的文档内容回答，如果文档中没有相关信息，请说明。
""")

    rag_chain = (
        retrieval_chain
        | context_chain
        | rag_prompt
        | llm
        | StrOutputParser()
    )

    # 测试 RAG 系统
    questions = [
        {"query": "什么是机器学习？"},
        {"query": "深度学习和机器学习有什么关系？"},
        {"query": "人工智能有哪些应用领域？"}
    ]

    for i, question in enumerate(questions, 1):
        print(f"\n问题{i}：{question['query']}")
        answer = rag_chain.invoke(question)
        print(f"回答{i}：{answer}")

# ============================================================================
# 12. 多模态处理链
# ============================================================================

def multimodal_processing_example():
    """多模态处理示例"""
    print("\n" + "=" * 60)
    print("12. 多模态处理链")
    print("=" * 60)

    llm = create_llm()

    # 模拟不同类型的输入处理
    def process_text_input(x):
        return {
            "type": "text",
            "content": x["content"],
            "word_count": len(x["content"].split()),
            "language": "中文" if any('\u4e00' <= char <= '\u9fff' for char in x["content"]) else "英文"
        }

    def process_image_input(x):
        # 模拟图像处理
        return {
            "type": "image",
            "filename": x["content"],
            "format": x["content"].split(".")[-1] if "." in x["content"] else "unknown",
            "description": f"这是一个{x['content']}文件"
        }

    def process_audio_input(x):
        # 模拟音频处理
        return {
            "type": "audio",
            "filename": x["content"],
            "duration": "未知",
            "description": f"这是一个音频文件：{x['content']}"
        }

    # 输入类型检测
    def detect_input_type(x):
        content = x["content"].lower()
        if content.endswith(('.jpg', '.png', '.gif', '.bmp')):
            return "image"
        elif content.endswith(('.mp3', '.wav', '.flac')):
            return "audio"
        else:
            return "text"

    # 创建多模态处理分支
    multimodal_branch = RunnableBranch(
        (lambda x: detect_input_type(x) == "image", RunnableLambda(process_image_input)),
        (lambda x: detect_input_type(x) == "audio", RunnableLambda(process_audio_input)),
        RunnableLambda(process_text_input)  # 默认文本处理
    )

    # 生成统一的分析报告
    analysis_prompt = PromptTemplate.from_template("""
请分析以下输入内容：

类型：{type}
内容：{content}
详细信息：{details}

请提供适合该类型内容的分析和建议。
""")

    def prepare_analysis_input(processed_data):
        details = {k: v for k, v in processed_data.items() if k not in ["type", "content"]}
        return {
            "type": processed_data["type"],
            "content": processed_data.get("content", processed_data.get("filename", "未知")),
            "details": str(details)
        }

    multimodal_chain = (
        multimodal_branch
        | RunnableLambda(prepare_analysis_input)
        | analysis_prompt
        | llm
        | StrOutputParser()
    )

    # 测试不同类型的输入
    test_inputs = [
        {"content": "这是一段中文文本，用于测试多模态处理系统。"},
        {"content": "photo.jpg"},
        {"content": "music.mp3"},
        {"content": "This is an English text for testing."}
    ]

    for i, input_data in enumerate(test_inputs, 1):
        print(f"\n输入{i}：{input_data['content']}")
        result = multimodal_chain.invoke(input_data)
        print(f"分析{i}：{result}")

# ============================================================================
# 13. 工作流编排系统
# ============================================================================

def workflow_orchestration_example():
    """工作流编排示例"""
    print("\n" + "=" * 60)
    print("13. 工作流编排系统")
    print("=" * 60)

    llm = create_llm()

    # 定义工作流步骤
    class WorkflowStep:
        def __init__(self, name: str, processor: Runnable):
            self.name = name
            self.processor = processor

    # 数据验证步骤
    def validate_data(x):
        data = x.get("data", {})
        errors = []

        if not data.get("title"):
            errors.append("标题不能为空")
        if not data.get("content"):
            errors.append("内容不能为空")
        if len(data.get("content", "")) < 10:
            errors.append("内容长度不能少于10字符")

        return {
            "data": data,
            "validation_errors": errors,
            "is_valid": len(errors) == 0,
            "step": "validation"
        }

    # 内容处理步骤
    def process_content(x):
        if not x["is_valid"]:
            return {**x, "step": "content_processing", "processed_content": None}

        content = x["data"]["content"]
        processed = {
            "original_length": len(content),
            "word_count": len(content.split()),
            "has_keywords": any(keyword in content.lower() for keyword in ["重要", "紧急", "优先"]),
            "processed_at": datetime.now().isoformat()
        }

        return {
            **x,
            "step": "content_processing",
            "processed_content": processed
        }

    # AI 分析步骤
    def ai_analysis(x):
        if not x["is_valid"] or not x["processed_content"]:
            return {**x, "step": "ai_analysis", "ai_result": None}

        analysis_prompt = f"""
分析以下内容：
标题：{x['data']['title']}
内容：{x['data']['content']}

请提供：
1. 内容摘要
2. 情感分析
3. 重要程度评级（1-5）
"""

        ai_result = (
            PromptTemplate.from_template(analysis_prompt)
            | llm
            | StrOutputParser()
        ).invoke({})

        return {
            **x,
            "step": "ai_analysis",
            "ai_result": ai_result
        }

    # 结果汇总步骤
    def summarize_results(x):
        summary = {
            "workflow_id": str(uuid.uuid4())[:8],
            "completed_at": datetime.now().isoformat(),
            "status": "success" if x["is_valid"] else "failed",
            "validation_errors": x.get("validation_errors", []),
            "content_stats": x.get("processed_content"),
            "ai_analysis": x.get("ai_result"),
            "original_data": x["data"]
        }

        return {**x, "step": "summary", "final_result": summary}

    # 构建工作流链
    workflow_chain = (
        RunnableLambda(validate_data)
        | RunnableLambda(process_content)
        | RunnableLambda(ai_analysis)
        | RunnableLambda(summarize_results)
    )

    # 测试工作流
    test_cases = [
        {
            "data": {
                "title": "重要会议通知",
                "content": "明天上午10点在会议室A召开重要的项目讨论会议，请所有相关人员准时参加。"
            }
        },
        {
            "data": {
                "title": "",  # 无效数据
                "content": "短内容"
            }
        },
        {
            "data": {
                "title": "日常工作报告",
                "content": "今天完成了数据分析任务，发现了一些有趣的趋势。团队协作良好，项目进展顺利。明天计划继续优化算法性能。"
            }
        }
    ]

    for i, test_case in enumerate(test_cases, 1):
        print(f"\n工作流{i}：")
        print(f"输入：{test_case['data']}")

        result = workflow_chain.invoke(test_case)
        final_result = result["final_result"]

        print(f"工作流ID：{final_result['workflow_id']}")
        print(f"状态：{final_result['status']}")

        if final_result["validation_errors"]:
            print(f"验证错误：{final_result['validation_errors']}")

        if final_result["content_stats"]:
            print(f"内容统计：{final_result['content_stats']}")

        if final_result["ai_analysis"]:
            print(f"AI分析：{final_result['ai_analysis']}")

# ============================================================================
# 14. 实时数据处理管道
# ============================================================================

def realtime_pipeline_example():
    """实时数据处理管道示例"""
    print("\n" + "=" * 60)
    print("14. 实时数据处理管道")
    print("=" * 60)

    llm = create_llm()

    # 模拟实时数据源
    def generate_realtime_data():
        import random
        data_types = ["user_action", "system_event", "error_log", "performance_metric"]

        return {
            "timestamp": datetime.now().isoformat(),
            "type": random.choice(data_types),
            "user_id": f"user_{random.randint(1000, 9999)}",
            "value": random.randint(1, 100),
            "message": f"随机事件消息 {random.randint(1, 1000)}"
        }

    # 数据过滤器
    def filter_data(x):
        # 过滤掉低价值数据
        if x["value"] < 20:
            return None
        return x

    # 数据增强
    def enrich_data(x):
        if x is None:
            return None

        # 添加计算字段
        x["priority"] = "high" if x["value"] > 80 else "medium" if x["value"] > 50 else "low"
        x["category"] = x["type"].replace("_", " ").title()
        x["processed_at"] = datetime.now().isoformat()

        return x

    # 异常检测
    def detect_anomalies(x):
        if x is None:
            return None

        # 简单的异常检测逻辑
        is_anomaly = (
            x["type"] == "error_log" or
            x["value"] > 95 or
            "error" in x["message"].lower()
        )

        x["is_anomaly"] = is_anomaly
        x["alert_level"] = "critical" if is_anomaly else "normal"

        return x

    # AI 分析（仅对异常数据）
    def ai_analyze_if_needed(x):
        if x is None or not x.get("is_anomaly"):
            return x

        analysis_prompt = f"""
分析以下异常事件：
类型：{x['type']}
用户：{x['user_id']}
数值：{x['value']}
消息：{x['message']}
优先级：{x['priority']}

请提供：
1. 可能的原因
2. 建议的处理方案
3. 风险评估
"""

        try:
            ai_analysis = (
                PromptTemplate.from_template(analysis_prompt)
                | llm
                | StrOutputParser()
            ).invoke({})
            x["ai_analysis"] = ai_analysis
        except Exception as e:
            x["ai_analysis"] = f"AI分析失败：{e}"

        return x

    # 构建实时处理管道
    pipeline = (
        RunnableLambda(filter_data)
        | RunnableLambda(enrich_data)
        | RunnableLambda(detect_anomalies)
        | RunnableLambda(ai_analyze_if_needed)
    )

    # 模拟实时数据流处理
    print("开始实时数据处理...")
    processed_count = 0
    anomaly_count = 0

    for i in range(10):  # 处理10条数据
        raw_data = generate_realtime_data()
        print(f"\n原始数据{i+1}：{raw_data}")

        result = pipeline.invoke(raw_data)

        if result is not None:
            processed_count += 1
            print(f"处理结果：优先级={result['priority']}, 异常={result['is_anomaly']}")

            if result.get("is_anomaly"):
                anomaly_count += 1
                print(f"异常分析：{result.get('ai_analysis', '无')}")
        else:
            print("数据被过滤")

    print(f"\n处理统计：")
    print(f"总处理数据：{processed_count}")
    print(f"检测到异常：{anomaly_count}")

# 运行专业示例
def run_professional_examples():
    """运行专业应用示例"""
    print("LCEL 专业应用场景示例")
    print("=" * 60)

    try:
        rag_system_example()
        multimodal_processing_example()
        workflow_orchestration_example()
        realtime_pipeline_example()

        print("\n" + "=" * 60)
        print("所有专业示例运行完成！")
        print("=" * 60)

    except Exception as e:
        print(f"运行出错：{e}")

if __name__ == "__main__":
    run_professional_examples()

LCEL 最佳实践总结
1. 选择合适的组合方式
- 简单链：使用 | 操作符
- 并行处理：使用 RunnableParallel
- 条件分支：使用 RunnableBranch
- 数据传递：使用 RunnablePassthrough
2. 性能优化
- 利用批量处理：batch() 和 abatch()
- 使用流式处理：stream() 和 astream()
- 合理使用并行：避免不必要的串行处理
3. 错误处理
- 实现重试机制
- 提供备用处理链
- 优雅的错误降级
4. 可维护性
- 模块化设计：将复杂逻辑拆分为小的 Runnable
- 清晰的数据流：使用 RunnablePassthrough.assign 管理状态
- 充分的日志和监控
5. 扩展性
- 动态链构建：根据配置构建不同的处理链
- 插件化架构：易于添加新的处理步骤
- 标准化接口：所有组件都实现 Runnable 接口
LCEL 为构建复杂的 AI 应用提供了强大而灵活的框架，通过合理使用这些模式，可以构建出高性能、可维护的应用系统。